# The objectives of this jupyter notebook (prepare_modeling_data_part1.ipynb):

1. Obtain flood tables for Ghana from "GLOBAL_FLOOD_DB/MODIS_EVENTS/V1" (data source: Google Earth Engine).


2. Calculate the "start time", "end time" and "duration" of each flood from the 'system:index' column in the original table, and add the calculated values to the table as corresponding columns. The values are added to the table as corresponding columns.


3. Save this table data as (Historical_Ghana_flood_info_table.csv) file.


**Note1: When you try to re-run the code below, be sure to redefine the path to the data file to suit your current environment.**

**Note2: It is recommended to use Google colab to run the following code, making sure you have permission to use it from Google Earth Engine.**

### Import Some necessary packages and autheticate the earthengine to be ready.

In [1]:
!earthengine authenticate
from google.colab import drive
drive.mount('/content/gdrive')
import ee
import pandas as pd
import re

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=kTv9CnX_zzx2-NuCsrNxnh5DUV3rY_Uma19A6hBamhs&tc=UbB_RDvC3sj6qAg2Nm720vTKQ2axSkii-qIDyuwLbyU&cc=FCYsB1HtzUvsCQstLhXJvBI2P4wu5ifewtH1cycG5K8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BU0npnyg8a_kEDvNUJxipJki3AvjJzbaiH3iEeoCH-YCB5dkK4bhqM

Successfully saved authorization token.
Mounted at /content/gdrive


### For Objective 1: Obtain flood tables for Ghana from "GLOBAL_FLOOD_DB/MODIS_EVENTS/V1".

In [2]:
# Initialize the Earth Engine module.
ee.Initialize()

# Load global floods data
global_flood_data = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1") # contains 913 flood events images
# Check how many imgs from this database
global_flood_num_images = global_flood_data.size().getInfo()
print("Number of flood (images/events) in global flood database (all of the world): ", global_flood_num_images)
print('-----------------------------------------------------------------------')

# Filtering of flood data using Ghana's country name and ISO country code
ghana_flood_data = global_flood_data.filter(
    ee.Filter.Or(
        ee.Filter.stringContains('countries', 'Ghana'),
        ee.Filter.stringContains('cc', 'GHA')
    )
)
ghana_flood_num_images = ghana_flood_data.size().getInfo()
print("Number of flood (images/events) in ghana: ", ghana_flood_num_images)
print('-----------------------------------------------------------------------')

# Converting flood datasets to lists
ghana_flood_data_list = ghana_flood_data.toList(ghana_flood_data.size())

# Create an empty DataFrame
ghana_flood_data_df = pd.DataFrame()

# Iterate through the list, adding each image's metadata(Info) to the DataFrame
for i in range(ghana_flood_data_list.size().getInfo()):
    image = ee.Image(ghana_flood_data_list.get(i))
    info = image.getInfo()
    properties = info['properties']
    # Use pd.Series to make properties as a single-row DataFrame
    properties_df = pd.DataFrame(properties, index=[0])
    # Use pd.concat to make properties_df append to ghana_flood_data_df
    ghana_flood_data_df = pd.concat([ghana_flood_data_df, properties_df], ignore_index=True)
ghana_flood_data_df

Number of flood (images/events) in global flood database (all of the world):  913
-----------------------------------------------------------------------
Number of flood (images/events) in ghana:  11
-----------------------------------------------------------------------


,dfo_centroid_y,dfo_main_cause,gfd_country_name,dfo_centroid_x,glide_index,slope_threshold,dfo_severity,system:footprint,threshold_b1b2,dfo_displaced,...,countries,dfo_other_country,system:time_start,dfo_dead,gfd_country_code,threshold_type,threshold_b7,system:asset_size,system:index,otsu_sample_res
0,14.211698,Heavy rain,"['MAURITANIA', 'ALGERIA', 'MALI', 'BURKINA FAS...",-3.102020,NA,5,1.0,NaN,0.700,10000,...,"Mali, Mauritania, Algeria, Guinea, Burkina Fas...",NA,1060473600000,31,"['MR', 'AG', 'ML', 'UV', 'TO', 'GH', 'IV', 'GV...",standard,675.00,33272798,DFO_2320_From_20030810_to_20031019,NaN
1,6.148203,Heavy rain,"['TOGO', 'GHANA', 'BENIN', 'NIGERIA']",1.223784,NA,5,2.0,NaN,0.852,300,...,"Ghana, Nigeria, Togo, Benin",NA,1097107200000,0,"['TO', 'GH', 'BN', 'NI']",otsu,631.55,2685194,DFO_2576_From_20041007_to_20041104,231.66
2,14.421787,Heavy rain,"['MAURITANIA', 'ALGERIA', 'MALI', 'BURKINA FAS...",-1.021340,FL-2006-000120-BFA,5,1.0,NaN,0.700,70000,...,"Mali, Mauritania, Algeria, Chad, Guinea, Burki...",Niger,1154563200000,4,"['MR', 'AG', 'ML', 'UV', 'TO', 'GH', 'IV', 'GV...",standard,675.00,49175985,DFO_2947_From_20060803_to_20061011,NaN
3,10.292106,Heavy rain,"['TOGO', 'GHANA', 'BENIN', 'NIGERIA', 'CAMEROO...",11.748370,FL-2007-000123-NGA,5,1.0,NaN,0.700,140000,...,"Chad, Ghana, Nigeria, Cameroon, Niger, Togo, B...",NA,1186185600000,101,"['TO', 'GH', 'BN', 'NI', 'CM', 'CD', 'NG']",standard,675.00,19615837,DFO_3151_From_20070804_to_20071021,NaN
4,13.037658,Heavy rain,"['MAURITANIA', 'ALGERIA', 'MALI', 'BURKINA FAS...",0.612005,FL-2007-000132-BFA,5,1.5,NaN,0.700,540000,...,"Mali, Chad, Guinea, Burkina Faso, Ivory Coast,...",,1185408000000,153,"['MR', 'AG', 'ML', 'UV', 'TO', 'GH', 'IV', 'GV...",standard,675.00,39504930,DFO_3166_From_20070726_to_20071010,NaN
5,16.454005,Heavy Rain,"['Western Sahara', 'MAURITANIA', 'MOROCCO', 'A...",2.936762,NA,5,1.0,NaN,0.700,300000,...,"Mali, Mauritania, Chad, Western Sahara, Guinea...",NA,1249862400000,12,"['WI', 'MR', 'MO', 'AG', 'ML', 'UV', 'TO', 'GH...",standard,675.00,75387037,DFO_3534_From_20090810_to_20090902,NaN
6,5.543563,Heavy Rain,"['TOGO', 'GHANA', 'BENIN', 'NIGERIA', 'CAMEROON']",6.509699,NA,5,1.0,NaN,0.700,5000,...,"Ghana, Nigeria, Togo, Benin",NA,1255996800000,0,"['TO', 'GH', 'BN', 'NI', 'CM']",standard,675.00,3520983,DFO_3555_From_20091020_to_20091026,NaN
7,7.931620,Heavy Rain,"['MALI', 'BURKINA FASO', 'TOGO', 'GHANA', ""COT...",-0.981038,NA,5,1.0,NaN,0.700,0,...,"Burkina Faso, Ivory Coast, Ghana",NA,1277164800000,30,"['ML', 'UV', 'TO', 'GH', 'IV', 'BN', 'UU']",standard,675.00,4488318,DFO_3663_From_20100622_to_20100626,NaN
8,8.279999,Heavy Rain,"['BURKINA FASO', 'TOGO', 'GHANA', ""COTE D'IVOI...",-0.809999,NA,5,1.5,NaN,0.700,2600,...,"Burkina Faso, Ghana, Togo",NA,1288915200000,0,"['UV', 'TO', 'GH', 'IV', 'BN', 'UU']",standard,675.00,1919307,DFO_3747_From_20101105_to_20101111,NaN
9,5.584658,Torrential Rain,"['GHANA', ""COTE D'IVOIRE""]",-0.945865,,5,1.0,NaN,0.787,43000,...,"Ivory Coast, Ghana",,1319587200000,14,"['GH', 'IV']",otsu,645.45,1352536,DFO_3872_From_20111026_to_20111029,231.66


### For Objective 2: Calculate the "start time", "end time" and "duration" of each flood and updated the original table.

In [3]:
# Extract the start and end dates from the 'system:index' column
ghana_flood_data_df['Start_Date'] = ghana_flood_data_df['system:index'].apply(lambda x: pd.to_datetime(re.search('From_(.*)_to', x).group(1)))
ghana_flood_data_df['End_Date'] = ghana_flood_data_df['system:index'].apply(lambda x: pd.to_datetime(re.search('to_(.*)', x).group(1)))

# Calculate the difference between End_Date and Start_Date in days
ghana_flood_data_df['Duration_Days'] = (ghana_flood_data_df['End_Date'] - ghana_flood_data_df['Start_Date']).dt.days

# Sort the table by start time and show only certain rows['id','Start_Date','End_Date','Duration_Days']
ghana_flood_data_df = ghana_flood_data_df.sort_values(by='Start_Date')
ghana_flood_data_df[['id','Start_Date','End_Date','Duration_Days','dfo_main_cause', 'dfo_displaced', "dfo_dead"]]

,id,Start_Date,End_Date,Duration_Days,dfo_main_cause,dfo_displaced,dfo_dead
0,2320,2003-08-10,2003-10-19,70,Heavy rain,10000,31
1,2576,2004-10-07,2004-11-04,28,Heavy rain,300,0
2,2947,2006-08-03,2006-10-11,69,Heavy rain,70000,4
4,3166,2007-07-26,2007-10-10,76,Heavy rain,540000,153
3,3151,2007-08-04,2007-10-21,78,Heavy rain,140000,101
5,3534,2009-08-10,2009-09-02,23,Heavy Rain,300000,12
6,3555,2009-10-20,2009-10-26,6,Heavy Rain,5000,0
7,3663,2010-06-22,2010-06-26,4,Heavy Rain,0,30
8,3747,2010-11-05,2010-11-11,6,Heavy Rain,2600,0
9,3872,2011-10-26,2011-10-29,3,Torrential Rain,43000,14


### For Objective 3: Save this table data as (Historical_Ghana_flood_info_table.csv) file.

In [4]:
# save this above table to our gdrive
file_path = "/content/gdrive/MyDrive/irp_project_111/data/Historical_Ghana_Flood_info_table.csv"
ghana_flood_data_df.to_csv(file_path, index=False)